In [1]:
import cfbd
from cfbd.rest import ApiException

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '04rcT5SKrEKA3iqNaGCjuvyH5M5oZ4LtP0bVHDXqnD/az808b5qIgJiq1F35vfsN'
configuration.api_key_prefix['Authorization'] = 'Bearer'

osuGames = []

In [2]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
lines_instance = cfbd.BettingApi(cfbd.ApiClient(configuration))
homeTeam = 'Ohio State'
games = api_instance.get_games(year=2014, team = homeTeam)

In [3]:
for game in games:
    gameId = game.id
    try:
        lines_response = lines_instance.get_lines(game_id=gameId)
        
    except ApiException as e:
        print("Exception when calling BettingApi->get_lines: %s\n" % e)


In [298]:
class Team:
    def __init__(self, name):
        self.name = name
        self.years = {}
        self.opponents = {}
        self.wins = 0
        self.losses = 0
        self.pushes=0
        
    def __str__(self):
        return f"{self.name} is {self.wins}-{self.losses}-{self.pushes}"

    def addYear(self, year, data):
        self.years[year] = data
        wins, loss, push = data.getNumericalDecision()
        self.wins += wins
        self.losses += loss
        self.pushes += push
        for k,v in data.games.items():
            if k in self.opponents:
                
            else:
                self.opponets = {"wins": 0, "losses": 0, "pushes": 0}
    
    def getYear(self, userYear):
        return self.years[userYear]
    
    def getOpponent(self, opp):
        res = {}
        numWins, numLoss, numPushed = 0, 0, 0
        for k,v in self.years.items():
            if opp in v.games:
                res[k] = v.games[opp].cover
                if v.games[opp].cover == 1:
                    numWins += 1
                elif v.games[opp].cover == -1:
                    numLoss += 1
                else:
                    numPushed += 1
        print(f"{self.name} is {numWins}-{numLoss}-{numPushed} aginst {opp}")
        

        return res




In [337]:
class Game:
    def __init__(self, opponent, osuPoints, oppPoints, spread):
        self.opponent = opponent
        self.osuPoints = osuPoints
        self.oppPoints = oppPoints
        self.spread = spread
        self.cover = self.osuCover()

    def __str__(self):
        return f"MyClass instance with variables: Opponent={self.opponent}, OSU={self.osuPoints}, OPP={self.oppPoints}, spread={self.spread}, cover={self.cover}"


    def osuCover(self):
        equated = self.osuPoints + self.spread
        if equated > self.oppPoints:
            return 1
        elif equated < self.oppPoints:
            return -1
        else:
            return 0


class Year:
    def __init__(self):
        self.winATS = 0
        self.lowATS = 0
        self.pushATS = 0
        self.wins = 0
        self.loss = 0
        self.push = 0
        self.games = {}

    def __str__(self):
        return f"{self.wins}-{self.loss}-{self.push}"
    
    def addGame(self, game):
        numWins = self.winATS * (len(self.games))
        numLoss = self.lowATS * (len(self.games))
        numPushed = self.pushATS * (len(self.games))
        self.games[game.opponent] = game
        if game.osuCover() == 1:
            numWins += 1
        elif game.osuCover() == -1:
            numLoss += 1
        else:
            numPushed += 1
        self.wins = numWins
        self.loss = numLoss
        self.push = numPushed
        self.winATS = numWins/(len(self.games))
        self.lowATS = numLoss/(len(self.games))
        self.pushATS = numPushed/(len(self.games))

    def getNumericalDecision(self):
        numWins = self.winATS * (len(self.games))
        numLoss = self.lowATS * (len(self.games))
        numPushed = self.pushATS * (len(self.games))
        return numWins, numLoss, numPushed
    


def getLines(lines_test, teamName):
    games = []
    #lines_test refers to lines of an entire szn
    #line is in week
    season = Year()
    for line in lines_test:
        for indLine in line.lines:
            if indLine.spread:
                spread = indLine.spread
        if line.home_team == teamName:
            test = Game(line.away_team, line.home_score, line.away_score, spread)
        else:
            test = Game(line.home_team, line.away_score, line.home_score, spread)
        season.addGame(test)
    return season
    



    

In [338]:
years = []
team = Team("Nebraska")
for i in range(2013, 2023):
   lines_test = lines_instance.get_lines(team=team.name, year = i) 
   print(lines_test)
   season = getLines(lines_test, team.name)
   team.addYear(i, season)

[{'away_conference': 'Mountain West',
 'away_score': 34,
 'away_team': 'Wyoming',
 'home_conference': 'Big Ten',
 'home_score': 37,
 'home_team': 'Nebraska',
 'id': 332430158,
 'lines': [{'away_moneyline': None,
            'formatted_spread': 'Nebraska -31',
            'home_moneyline': None,
            'over_under': 65.5,
            'over_under_open': None,
            'provider': 'teamrankings',
            'spread': -31.0,
            'spread_open': None},
           {'away_moneyline': None,
            'formatted_spread': 'Nebraska -31',
            'home_moneyline': None,
            'over_under': 65.5,
            'over_under_open': None,
            'provider': 'numberfire',
            'spread': -31.0,
            'spread_open': None},
           {'away_moneyline': None,
            'formatted_spread': 'Nebraska -21.5',
            'home_moneyline': None,
            'over_under': None,
            'over_under_open': None,
            'provider': 'consensus',
            's

In [340]:
print(team.getYear(2021))

4.0-7.0-1.0
